# imports/setup

In [36]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation
import io
from backtester import Backtester

In [37]:
import sys
print(sys.executable)


/Users/pranavtiwari/CodeProjects/IMC_Prosperity/prosperity-3/.venv/bin/python


In [38]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

In [39]:
def calculate_kelp_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_resin_fair(order_depth):
    return 10000

In [40]:
listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 50,
    'KELP': 50
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_resin_fair,
    "KELP": calculate_kelp_fair
}


In [41]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import jsonpickle
import numpy as np
import math


class Product:
    RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"

PARAMS = {
    Product.RESIN: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 10,
    },
    Product.KELP: {
        "take_width": 1,
        "clear_width": 0,
        "prevent_adverse": True,
        "adverse_volume": 15,
        'rolling_window_size': 10,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
}


class Trader:
    def __init__(self, params=None):
        if params is None:
            params = PARAMS
        self.params = params

        self.LIMIT = {Product.RESIN: 50, Product.KELP: 50}

    def take_best_orders(
        self,
        product: str,
        fair_value: int,
        take_width: float,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (int, int):
        position_limit = self.LIMIT[product]

        if len(order_depth.sell_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_ask_amount = -1 * order_depth.sell_orders[best_ask]

            if not prevent_adverse or abs(best_ask_amount) <= adverse_volume:
                if best_ask <= fair_value - take_width:
                    quantity = min(
                        best_ask_amount, position_limit - position
                    )  # max amt to buy
                    if quantity > 0:
                        orders.append(Order(product, best_ask, quantity))
                        buy_order_volume += quantity
                        order_depth.sell_orders[best_ask] += quantity
                        if order_depth.sell_orders[best_ask] == 0:
                            del order_depth.sell_orders[best_ask]

        if len(order_depth.buy_orders) != 0:
            best_bid = max(order_depth.buy_orders.keys())
            best_bid_amount = order_depth.buy_orders[best_bid]

            if not prevent_adverse or abs(best_bid_amount) <= adverse_volume:
                if best_bid >= fair_value + take_width:
                    quantity = min(
                        best_bid_amount, position_limit + position
                    )  # should be the max we can sell
                    if quantity > 0:
                        orders.append(Order(product, best_bid, -1 * quantity))
                        sell_order_volume += quantity
                        order_depth.buy_orders[best_bid] -= quantity
                        if order_depth.buy_orders[best_bid] == 0:
                            del order_depth.buy_orders[best_bid]

        return buy_order_volume, sell_order_volume

    def market_make(
        self,
        product: str,
        orders: List[Order],
        bid: int,
        ask: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (int, int):
        buy_quantity = self.LIMIT[product] - (position + buy_order_volume)
        if buy_quantity > 0:
            orders.append(Order(product, round(bid), buy_quantity))  # Buy order

        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)
        if sell_quantity > 0:
            orders.append(Order(product, round(ask), -sell_quantity))  # Sell order
        return buy_order_volume, sell_order_volume

    def clear_position_order(
        self,
        product: str,
        fair_value: float,
        width: int,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> List[Order]:
        position_after_take = position + buy_order_volume - sell_order_volume
        fair_for_bid = round(fair_value - width)
        fair_for_ask = round(fair_value + width)

        buy_quantity = self.LIMIT[product] - (position + buy_order_volume)
        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)

        if position_after_take > 0:
            # Aggregate volume from all buy orders with price greater than fair_for_ask
            clear_quantity = sum(
                volume
                for price, volume in order_depth.buy_orders.items()
                if price >= fair_for_ask
            )
            clear_quantity = min(clear_quantity, position_after_take)
            sent_quantity = min(sell_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_ask, -abs(sent_quantity)))
                sell_order_volume += abs(sent_quantity)

        if position_after_take < 0:
            # Aggregate volume from all sell orders with price lower than fair_for_bid
            clear_quantity = sum(
                abs(volume)
                for price, volume in order_depth.sell_orders.items()
                if price <= fair_for_bid
            )
            clear_quantity = min(clear_quantity, abs(position_after_take))
            sent_quantity = min(buy_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_bid, abs(sent_quantity)))
                buy_order_volume += abs(sent_quantity)

        return buy_order_volume, sell_order_volume

    def kelp_fair_value(self, order_depth: OrderDepth, traderObject) -> float:
        if len(order_depth.sell_orders) != 0 and len(order_depth.buy_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_bid = max(order_depth.buy_orders.keys())
            filtered_ask = [
                price
                for price in order_depth.sell_orders.keys()
                if abs(order_depth.sell_orders[price])
                >= self.params[Product.KELP]["adverse_volume"]
            ]
            filtered_bid = [
                price
                for price in order_depth.buy_orders.keys()
                if abs(order_depth.buy_orders[price])
                >= self.params[Product.KELP]["adverse_volume"]
            ]
            mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else None
            mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else None


            if mm_ask == None or mm_bid == None:
                if traderObject.get("kelp_prices", None) == None:
                    mmmid_price = (best_ask + best_bid) / 2
                else:
                    mmmid_price = traderObject["kelp_prices"][-1]
            else:
                mmmid_price = (mm_ask + mm_bid) / 2

            if traderObject.get('kelp_prices') != None:
                price_window = traderObject['kelp_prices'] + [mmmid_price]
                fair = sum(price_window) / len(price_window)
            else:
                price_window = [mmmid_price]
                fair = mmmid_price
            
            rolling_window_size = self.params[Product.KELP]['rolling_window_size']

            if len(price_window) > rolling_window_size:
                price_window.pop(0)

            traderObject['kelp_prices'] = price_window
            return fair
        return None

            # if traderObject.get("kelp_last_price", None) != None:
            #     last_price = traderObject["kelp_last_price"]
            #     last_returns = (mmmid_price - last_price) / last_price
            #     pred_returns = (
            #         last_returns * self.params[Product.kelp]["reversion_beta"]
            #     )
            #     fair = mmmid_price + (mmmid_price * pred_returns)
            # else:
            #     fair = mmmid_price
            # traderObject["kelp_last_price"] = mmmid_price
            # return fair
        # return None

    def take_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        take_width: float,
        position: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume = 0
        sell_order_volume = 0

        buy_order_volume, sell_order_volume = self.take_best_orders(
            product,
            fair_value,
            take_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
            prevent_adverse,
            adverse_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    def clear_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        clear_width: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume, sell_order_volume = self.clear_position_order(
            product,
            fair_value,
            clear_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    def make_orders(
        self,
        product,
        order_depth: OrderDepth,
        fair_value: float,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        disregard_edge: float,  # disregard trades within this edge for pennying or joining
        join_edge: float,  # join trades within this edge
        default_edge: float,  # default edge to request if there are no levels to penny or join
        manage_position: bool = False,
        soft_position_limit: int = 0,
        # will penny all other levels with higher edge
    ):
        orders: List[Order] = []
        asks_above_fair = [
            price
            for price in order_depth.sell_orders.keys()
            if price > fair_value + disregard_edge
        ]
        bids_below_fair = [
            price
            for price in order_depth.buy_orders.keys()
            if price < fair_value - disregard_edge
        ]

        best_ask_above_fair = min(asks_above_fair) if len(asks_above_fair) > 0 else None
        best_bid_below_fair = max(bids_below_fair) if len(bids_below_fair) > 0 else None

        ask = round(fair_value + default_edge)
        if best_ask_above_fair != None:
            if abs(best_ask_above_fair - fair_value) <= join_edge:
                ask = best_ask_above_fair  # join
            else:
                ask = best_ask_above_fair - 1  # penny

        bid = round(fair_value - default_edge)
        if best_bid_below_fair != None:
            if abs(fair_value - best_bid_below_fair) <= join_edge:
                bid = best_bid_below_fair
            else:
                bid = best_bid_below_fair + 1

        if manage_position:
            if position > soft_position_limit:
                ask -= 1
            elif position < -1 * soft_position_limit:
                bid += 1

        buy_order_volume, sell_order_volume = self.market_make(
            product,
            orders,
            bid,
            ask,
            position,
            buy_order_volume,
            sell_order_volume,
        )

        return orders, buy_order_volume, sell_order_volume

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}

        if Product.RESIN in self.params and Product.RESIN in state.order_depths:
            resin_position = (
                state.position[Product.RESIN]
                if Product.RESIN in state.position
                else 0
            )
            resin_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.RESIN,
                    state.order_depths[Product.RESIN],
                    self.params[Product.RESIN]["fair_value"],
                    self.params[Product.RESIN]["take_width"],
                    resin_position,
                )
            )
            resin_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.RESIN,
                    state.order_depths[Product.RESIN],
                    self.params[Product.RESIN]["fair_value"],
                    self.params[Product.RESIN]["clear_width"],
                    resin_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            resin_make_orders, _, _ = self.make_orders(
                Product.RESIN,
                state.order_depths[Product.RESIN],
                self.params[Product.RESIN]["fair_value"],
                resin_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.RESIN]["disregard_edge"],
                self.params[Product.RESIN]["join_edge"],
                self.params[Product.RESIN]["default_edge"],
                True,
                self.params[Product.RESIN]["soft_position_limit"],
            )
            result[Product.RESIN] = (
                resin_take_orders + resin_clear_orders + resin_make_orders
            )

        if Product.KELP in self.params and Product.KELP in state.order_depths:
            kelp_position = (
                state.position[Product.KELP]
                if Product.KELP in state.position
                else 0
            )
            kelp_fair_value = self.kelp_fair_value(
                state.order_depths[Product.KELP], traderObject
            )
            kelp_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.KELP,
                    state.order_depths[Product.KELP],
                    kelp_fair_value,
                    self.params[Product.KELP]["take_width"],
                    kelp_position,
                    self.params[Product.KELP]["prevent_adverse"],
                    self.params[Product.KELP]["adverse_volume"],
                )
            )
            kelp_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.KELP,
                    state.order_depths[Product.KELP],
                    kelp_fair_value,
                    self.params[Product.KELP]["clear_width"],
                    kelp_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            kelp_make_orders, _, _ = self.make_orders(
                Product.KELP,
                state.order_depths[Product.KELP],
                kelp_fair_value,
                kelp_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.KELP]["disregard_edge"],
                self.params[Product.KELP]["join_edge"],
                self.params[Product.KELP]["default_edge"],
            )
            result[Product.KELP] = (
                kelp_take_orders + kelp_clear_orders + kelp_make_orders
            )

        conversions = 1
        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData


In [42]:
market_data, trade_history = _process_data_('logs/empty_submission.log')
trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/rolling_kelp.log")
backtester.run()
print(backtester.pnl)

{'RAINFOREST_RESIN': 3080, 'KELP': 627.0}


`{'RAINFOREST_RESIN': 3010, 'KELP': 627.0}`

# backtest gridsearch

In [43]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [44]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = ";".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [45]:
listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 50,
    'KELP': 50
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_resin_fair,
    "KELP": calculate_kelp_fair
}


In [46]:
# trader v1

param_grid = {
    Product.RESIN: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [0.5],
        "volume_limit": [0],
        # for making
        "disregard_edge": [1],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4],
        'soft_position_limit': [10]
    },
    Product.KELP: {
        "take_width": [1, 2, 3],
        "clear_width": [-0.25, 0, 1],
        "prevent_adverse": [True],
        "adverse_volume": [15],
        'rolling_window_size': list(range(3, 21)),
        # for making
        "disregard_edge": [1],
        "join_edge": [3],
        "default_edge": [5],
        'soft_position_limit': [0]
    },
}

In [47]:
# trader v2

param_grid = {
    Product.RESIN: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [0.5],
        "volume_limit": [0],
        # for making
        "disregard_edge": [1],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4],
        'soft_position_limit': [10]
    },
    Product.KELP: {
        "take_width": [1, 2, 3],
        "clear_width": [-0.25, 0, 1],
        "prevent_adverse": [True],
        "adverse_volume": [15],
        'reversion_beta': [-0.18172393033850867],
        # for making
        "disregard_edge": [1],
        "join_edge": [3],
        "default_edge": [5],
        'soft_position_limit': [0]
    },
}

backtest_dir = "backtests/trader_v2/"

In [59]:
# trader v0 params


param_grid = {
    Product.RESIN: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [0],
        "volume_limit": [0],
        # for making
        "disregard_edge": [1],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4],
        'soft_position_limit': [10]
    },
    Product.KELP: {
        "take_width": [2],
        "clear_width": [1],
        "prevent_adverse": [True],
        "adverse_volume": [5, 10, 13, 14, 15, 16, 17, 20, 25, 30],
        'reversion_beta': [-0.18172393033850867],
        # for making
        "disregard_edge": [1],
        "join_edge": [0],
        "default_edge": [1]
    },
}

backtest_dir = "backtests/linregv1/"

### run

In [60]:
# from trader_v2 import Trader
from trader_v0 import Trader

trader = Trader()

# amethyst_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "RAINFOREST_RESIN")
# print("RAINFOREST_RESIN results:")
# for params, pnl in amethyst_results: 
#     print(params)
#     print(f"pnl: {pnl}")
#     print("="*80)

starfruit_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "KELP")
print("KELP results:")
for params, pnl in starfruit_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

Running backtests for KELP: 100%|██████████| 10/10 [00:07<00:00,  1.27backtest/s]

KELP results:
{'take_width': 2, 'clear_width': 1, 'prevent_adverse': True, 'adverse_volume': 5, 'reversion_beta': -0.18172393033850867, 'disregard_edge': 1, 'join_edge': 0, 'default_edge': 1}
pnl: 587.0
{'take_width': 2, 'clear_width': 1, 'prevent_adverse': True, 'adverse_volume': 10, 'reversion_beta': -0.18172393033850867, 'disregard_edge': 1, 'join_edge': 0, 'default_edge': 1}
pnl: 677.0
{'take_width': 2, 'clear_width': 1, 'prevent_adverse': True, 'adverse_volume': 13, 'reversion_beta': -0.18172393033850867, 'disregard_edge': 1, 'join_edge': 0, 'default_edge': 1}
pnl: 688.0
{'take_width': 2, 'clear_width': 1, 'prevent_adverse': True, 'adverse_volume': 14, 'reversion_beta': -0.18172393033850867, 'disregard_edge': 1, 'join_edge': 0, 'default_edge': 1}
pnl: 688.0
{'take_width': 2, 'clear_width': 1, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.18172393033850867, 'disregard_edge': 1, 'join_edge': 0, 'default_edge': 1}
pnl: 688.0
{'take_width': 2, 'clear_width': 1, '

## analyze

In [61]:
def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split(';', 1)
        params = dict(param.split('=') for param in params_str.split(';'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

KELP_take_width=2;clear_width=0.5;prevent_adverse=True;adverse_volume=10;reversion_beta=-0.18172393033850867;disregard_edge=1;join_edge=0;default_edge=1
KELP_take_width=2;clear_width=1;prevent_adverse=True;adverse_volume=15;reversion_beta=-0.18172393033850867;disregard_edge=1;join_edge=0;default_edge=1
KELP_take_width=2;clear_width=1;prevent_adverse=True;adverse_volume=13;reversion_beta=-0.18172393033850867;disregard_edge=1;join_edge=0;default_edge=1
KELP_take_width=0;clear_width=0;prevent_adverse=True;adverse_volume=10;reversion_beta=-0.18172393033850867;disregard_edge=1;join_edge=0;default_edge=1
KELP_take_width=1;clear_width=-0.25;prevent_adverse=True;adverse_volume=20;reversion_beta=-0.18172393033850867;disregard_edge=1;join_edge=0;default_edge=1
KELP_take_width=4;clear_width=0;prevent_adverse=True;adverse_volume=20;reversion_beta=-0.18172393033850867;disregard_edge=1;join_edge=0;default_edge=1
KELP_take_width=3;clear_width=-0.25;prevent_adverse=True;adverse_volume=15;reversion_bet

In [62]:
sorted_kelp_results = sorted(starfruit_results, key=lambda x: x[1], reverse=True)

In [63]:
sorted_kelp_results[0:100]

[({'take_width': 2,
   'clear_width': 1,
   'prevent_adverse': True,
   'adverse_volume': 13,
   'reversion_beta': -0.18172393033850867,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 1},
  688.0),
 ({'take_width': 2,
   'clear_width': 1,
   'prevent_adverse': True,
   'adverse_volume': 14,
   'reversion_beta': -0.18172393033850867,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 1},
  688.0),
 ({'take_width': 2,
   'clear_width': 1,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.18172393033850867,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 1},
  688.0),
 ({'take_width': 2,
   'clear_width': 1,
   'prevent_adverse': True,
   'adverse_volume': 16,
   'reversion_beta': -0.18172393033850867,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 1},
  688.0),
 ({'take_width': 2,
   'clear_width': 1,
   'prevent_adverse': True,
   'adverse_volume': 17,
   'reversion_beta': -0.18172393033850867,
   'disrega